In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv
import matplotlib as plt
from tqdm import tqdm

import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input/copy-of-used-cars-price-prediction-ds18'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_percentage_error


from catboost import CatBoostRegressor
from catboost import Pool, cv

import warnings
warnings.filterwarnings("ignore")

# открываем и смотрим данные

In [ ]:
df_train= pd.read_csv('/kaggle/input/copy-of-used-cars-price-prediction-ds18/train.csv')
df_test=pd.read_csv('/kaggle/input/copy-of-used-cars-price-prediction-ds18/test.csv')

In [ ]:
def data_analis (df):
    i=df.info()
    h=df.head()
    ds=df.describe()
    dp=df.duplicated().sum()
    null= df.isna().sum()
    return print(i),print(),print(h),print(), print(ds),print('дубликаты:',dp),print(),print('пропуски:',null)

In [ ]:
data_analis(df_train)

In [ ]:
print(df_train.shape)

# переработка данных

In [ ]:
df_train['make']=df_train['make'].dropna()
df_train['model']=df_train['model'].dropna()
df_train['body']=df_train['body'].dropna()
df_train['trim']=df_train['trim'].dropna()
df_train['color']=df_train['color'].dropna()
df_train['interior']=df_train['interior'].dropna()


In [ ]:
df_train['saledate']=pd.to_datetime(df_train['saledate'],utc=True)

In [ ]:
df_test['saledate'].info()

In [ ]:
df_train['make'].unique()

In [ ]:
df_train['make']=df_train['make'].str.lower()
df_train['make'].unique()

In [ ]:
mers_after=['mercedes','mercedes-b']
mers_befor='mercedes-benz'

dod_after=['ram','dot']
dod_befor='dodge'

lr_after=['landrover','land rover']
lr_before='land_rover'

vw_after='vw'
vw_before='volkswagen'

df_train['make']= df_train['make'].replace(mers_after,mers_befor)
df_train['make']= df_train['make'].replace(dod_after,dod_befor)
df_train['make']= df_train['make'].replace(vw_after,vw_before)
df_train['make']= df_train['make'].replace(lr_after,lr_before)

In [ ]:
df_train['make'].unique()

In [ ]:
df_train['model']=df_train['model'].str.lower()

In [ ]:
df_train['trim']=df_train['trim'].str.lower()

In [ ]:
df_train['body']=df_train['body'].str.lower()
df_train['body']=df_train['body'].replace('koup','coupe')

df_train['body']=df_train['body'].replace('supercab','super cabe')
df_train['body']=df_train['body'].replace('xtracab','xtra cab')
df_train['body']=df_train['body'].replace('regular-cab','regular cab')

df_train['body']=df_train['body'].fillna('suv')
df_train['body'].unique()

# смотрим корреляции

In [ ]:
sns.heatmap(df_train.corr(),annot=True)

In [ ]:
ax=sns.barplot(data=df_train,x='year',y='condition')
ax.figure.set_size_inches(20,10)

In [ ]:
ax=sns.barplot(data=df_train,x='condition',y='odometer')
ax.figure.set_size_inches(20,10)

In [ ]:
df_train['condition']=df_train['condition'].fillna(df_train['condition'].corr(df_train['odometer']))

In [ ]:
df_train['transmission']=df_train['transmission'].fillna('automatic')

In [ ]:
df_train.isna().sum()

In [ ]:
ax=sns.barplot(data=df_train,x='condition',y='odometer')
ax.figure.set_size_inches(20,10)

In [ ]:
ax=sns.barplot(data=df_train,x='sellingprice',y='make')
ax.figure.set_size_inches(10,10)

In [ ]:
ax=sns.barplot(data=df_train,x='transmission',y='sellingprice' )
ax.figure.set_size_inches(10,10)

# Подготовка данных для обучения модели

In [ ]:
df_train=df_train.drop(['vin','saledate','seller' ],axis=1)

In [ ]:
train,test=train_test_split(df_train,random_state=3826)

In [ ]:
cat_features=list(train.select_dtypes(include='object').columns)
cat_features

In [ ]:
train[cat_features]= train[cat_features].fillna('not_info')
test[cat_features]= test[cat_features].fillna('not_info')

In [ ]:
train.head()

In [ ]:
print(test.shape)
print(train.shape)

In [ ]:
x=['make', 'model', 'body', 'transmission','year','condition','odometer','trim','interior','state','color']
y=['sellingprice']

In [ ]:
train_data= Pool(data=train[x],
                label=train[y],
                 cat_features=cat_features)
test_data= Pool(data=test[x],
                label=test[y],
                 cat_features=cat_features)

In [ ]:
params={'eval_metric':'MAPE',
       'loss_function':'RMSE',
        'task_type':'GPU',
       'learning_rate':0.1,
       'random_seed':3826,
       'verbose':100,
       'iterations':6000}

# обучение и проверка модели

In [ ]:
cv_data=cv(params= params,
          pool=train_data,
          fold_count=5,
          shuffle=True,
          partition_random_seed=0,
          stratified=True,
          verbose=False,
          early_stopping_rounds=200,)

In [ ]:
cv_data[cv_data['test-MAPE-mean']==cv_data['test-MAPE-mean'].min()]

In [ ]:
model=CatBoostRegressor(**params)

In [ ]:
model.fit(train_data)

In [ ]:
test['y_pred'] = model.predict(test_data)

In [ ]:
mean_absolute_percentage_error(test['sellingprice'],test['y_pred'])

In [ ]:
cv_data[cv_data['test-MAPE-mean'] == cv_data['test-MAPE-mean'].min()]

# проверка модели на тестовых данных

In [ ]:
df_test_1=df_test.drop(['vin','saledate','seller' ],axis=1)

In [ ]:
df_test_1['transmission']=df_test_1['transmission'].fillna('automatic')

In [ ]:
cat_features_test=list(df_test_1.select_dtypes(include='object').columns)
cat_features_test 

In [ ]:
df_test_1[cat_features_test]= df_test_1[cat_features_test].fillna('not_info')

In [ ]:
mers_after=['mercedes','mercedes-b']
mers_befor='mercedes-benz'

dod_after=['ram','dot']
dod_befor='dodge'

lr_after=['landrover','land rover']
lr_before='land_rover'

vw_after='vw'
vw_before='volkswagen'

df_test_1['make']= df_test_1['make'].replace(mers_after,mers_befor)
df_test_1['make']= df_test_1['make'].replace(dod_after,dod_befor)
df_test_1['make']= df_test_1['make'].replace(vw_after,vw_before)
df_test_1['make']= df_test_1['make'].replace(lr_after,lr_before)

In [ ]:
df_test_1['body']=df_test_1['body'].str.lower()
df_test_1['body']=df_test_1['body'].replace('koup','coupe')

df_test_1['body']=df_test_1['body'].replace('supercab','super cabe')
df_test_1['body']=df_test_1['body'].replace('xtracab','xtra cab')
df_test_1['body']=df_test_1['body'].replace('regular-cab','regular cab')

df_test_1['body']=df_test_1['body'].fillna('suv')
df_test_1['body'].unique()

In [ ]:
df_test_1['trim']=df_test_1['trim'].str.lower()

In [ ]:
df_test_1['make']=df_test_1['make'].str.lower()

In [ ]:
df_test_1['model']=df_test_1['model'].str.lower()

In [ ]:
sample_submission=pd.read_csv('/kaggle/input/copy-of-used-cars-price-prediction-ds18/sample_submission.csv')

In [ ]:
sample_submission = pd.DataFrame()
sample_submission['vin'] = df_test['vin']
sample_submission['sellingprice'] = model.predict(df_test_1[x])

In [ ]:
df_test['sellingprice']=model.predict(df_test_1[x])

In [ ]:
ax=sns.barplot(data=df_test,x='sellingprice',y='make')
ax.figure.set_size_inches(10,10)

In [ ]:
output = pd.DataFrame(sample_submission)
output.to_csv('submission.csv',index=False)

In [ ]:
k=pd.read_csv('/kaggle/working/submission.csv')
print(k)